In [7]:
import numpy as np
import pandas as pnd
import geopandas as gp
import requests as rq
import json
import geojson
import csv
import pathlib
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# current_folder = pathlib.Path(__file__).parent.absolute()
# main_folder = current_folder.parent
# training_data_folder = main_folder / 'training_data'
# data_folder = main_folder / 'data'

In [11]:
url = 'https://api.digitransit.fi/routing/v1/routers/hsl/index/graphql'
headers = { "Content-Type": "application/graphql" }
routes = pnd.read_csv('workfiles/district_pairs_pred.csv')
centroids = gp.read_file('data/Base_district_centroids_edited.geojson')
centroids = centroids.to_crs("EPSG:4326")
centroids['tunnus'] = pnd.to_numeric(centroids['tunnus'])
centroids

,tietopalvelu_id,aluejako,tunnus,nimi_fi,nimi_se,tyyppi,pa,paivitetty_tietopalveluun,geometry
0,8,PERUSPIIRI,104,TAKA-TÖÖLÖ,BORTRE TÖLÖ,Maa-alue,1958322.4,2020-10-16,POINT (24.92162 60.18412)
1,11,PERUSPIIRI,201,REIJOLA,GREJUS,Maa-alue,4520378.5,2020-10-16,POINT (24.90067 60.19480)
2,18,PERUSPIIRI,202,MUNKKINIEMI,MUNKSNÄS,Maa-alue,4682689.5,2020-10-16,POINT (24.86839 60.19767)
3,21,PERUSPIIRI,506,JAKOMÄKI,JAKOBACKA,Maa-alue,1949469.5,2020-10-16,POINT (25.07831 60.25910)
4,28,PERUSPIIRI,303,VALLILA,VALLGÅRD,Maa-alue,2625569.8,2020-10-16,POINT (24.96273 60.19532)
5,33,PERUSPIIRI,102,ULLANLINNA,ULRIKASBORG,Maa-alue,6289574.5,2020-10-16,POINT (24.94423 60.15871)
6,59,PERUSPIIRI,701,VARTIOKYLÄ,BOTBY,Maa-alue,7093867.0,2020-10-16,POINT (25.09002 60.21323)
7,60,PERUSPIIRI,101,VIRONNIEMI,ESTNÄS,Maa-alue,2043261.0,2020-10-16,POINT (24.95391 60.17122)
8,65,PERUSPIIRI,204,PITÄJÄNMÄKI,SOCKENBACKA,Maa-alue,6659828.5,2020-10-16,POINT (24.85510 60.22825)
9,69,PERUSPIIRI,404,OULUNKYLÄ,ÅGGELBY,Maa-alue,4344792.5,2020-10-16,POINT (24.96510 60.23098)


In [12]:
routes = routes.merge(centroids, left_on="start", right_on="tunnus")
routes.drop(['tietopalvelu_id', 'aluejako', 'tunnus', 'nimi_fi','nimi_se','tyyppi','pa','paivitetty_tietopalveluun'], axis=1, inplace=True)
routes = gp.GeoDataFrame(routes)
routes['start_lon'] = routes['geometry'].x
routes['start_lat'] = routes['geometry'].y
routes.drop(['geometry'], axis=1, inplace=True)
routes = routes.merge(centroids, left_on="end", right_on="tunnus")
routes.drop(['tietopalvelu_id', 'aluejako', 'tunnus', 'nimi_fi','nimi_se','tyyppi','pa','paivitetty_tietopalveluun'], axis=1, inplace=True)
routes['end_lon'] = routes['geometry'].x
routes['end_lat'] = routes['geometry'].y
routes.drop(['geometry'], axis=1, inplace=True)
routes = routes.sort_values(['start', 'end'], ignore_index=True)
routes

,start,end,start_lon,start_lat,end_lon,end_lat
0,205,101,24.880285,60.248621,24.953912,60.171221
1,205,102,24.880285,60.248621,24.944232,60.158714
2,205,103,24.880285,60.248621,24.918583,60.165133
3,205,104,24.880285,60.248621,24.921618,60.184116
4,205,105,24.880285,60.248621,24.872722,60.157297
...,...,...,...,...,...,...
358,801,603,25.196369,60.262256,25.051839,60.163059
359,801,701,25.196369,60.262256,25.090020,60.213227
360,801,702,25.196369,60.262256,25.062962,60.221196
361,801,703,25.196369,60.262256,25.088125,60.236207


In [13]:
s = rq.Session()
retries = Retry(total=5,
                backoff_factor=1,
                status_forcelist=[ 500, 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))
start_time = time.perf_counter()
distances = []
for index, row in routes.iterrows():
  body = '{plan(fromPlace: "::' + str(row['start_lat']) + ',' + str(row['start_lon']) + '", toPlace: "::' + str(row['end_lat']) + ',' + str(row['end_lon']) + '", numItineraries: 1, transportModes: {mode: BICYCLE}) {itineraries{duration legs {distance legGeometry {length points}}}}}'
  request = rq.post(url, data = body, headers = headers)
  if ('data' in request.json()) and (len(request.json()['data']['plan']['itineraries']) > 0): 
    itinerary = request.json()['data']['plan']['itineraries'][0]
    distance = 0
    for leg in itinerary['legs']:
      distance = distance + leg['distance']
    distances.append(distance)
  else:
    distances.append(0)
distances

[11485.119999999999,
 11981.892999999993,
 10660.037999999997,
 8650.195,
 13294.888000000008,
 6967.379000000001,
 7481.999999999998,
 3576.6059999999998,
 3089.2819999999983,
 10568.629,
 8773.417999999994,
 8801.283999999992,
 6634.893000000001,
 7025.007999999999,
 4017.2850000000008,
 3514.620999999999,
 4176.145,
 6339.723999999997,
 5108.505999999999,
 9630.659000000009,
 6867.188000000001,
 9200.418999999998,
 8352.094000000001,
 11192.721000000003,
 12957.902999999998,
 11727.82999999999,
 13804.129000000012,
 17759.104999999992,
 14870.874000000003,
 12790.601000000004,
 13593.249000000003,
 18867.204999999998,
 22313.862999999994,
 12007.028,
 13900.663999999993,
 12689.105000000001,
 10663.330000000002,
 15711.428000000007,
 9205.092999999999,
 10632.045999999991,
 6442.218999999996,
 6674.199000000002,
 4080.114,
 10840.806999999997,
 10165.038,
 9105.065,
 7657.424000000002,
 7051.832,
 4715.837999999998,
 2878.464000000001,
 4621.057000000002,
 2608.769,
 7381.8160000000